# gAIden - Jogando Ninja Gaiden via Deep Q-Learning

## 0 - Criando nosso ambiente

In [ ]:
#Importamos todas as bibliotecas que utilizaremos:
import tensorflow as tf  
import numpy as np
import retro
from skimage import transform # Help us to preprocess the frames
from skimage.color import rgb2gray # Help us to gray our frames
import matplotlib.pyplot as plt # Display graphs
from collections import deque# Ordered collection with ends
import random
import warnings 

In [ ]:
#Criamos nosso ambiente:
env = retro.make(game='NinjaGaiden-Nes') 

In [ ]:
#Estudamos os tamanhos de nossos frames e ações:
print('O tamanho de cada frame é:', env.observation_space)
print('Nosso espaço de ações tem tamanho :', env.action_space.n)

#Fazemos um one-hot-encoding para nossas ações:
possible_actions = np.array(np.identity(env.action_space.n,dtype=int).tolist())

## 1 - Processamento e Stack de frames

In [ ]:
#Definimos a função de pré-processamento de frames:

def preprocess(frame):
    preto_e_branco=rgb2gray(frame)    #converte o frame em preto e branco
    corte=preto_e_branco[45:-12,6:-12] #corta pedaços da tela que não estamos usando
    norm=corte/255                    #normaliza o frame
    frame_processado=transform.resize(norm,[110,84]) 
    
    return frame_processado

Antes de processarmos nossa imagem, ela era:

In [ ]:
from matplotlib.pyplot import imshow
print(imshow(rgb2gray(env.reset())))

Depois do pré-processamento, ela se torna:

In [ ]:
print(imshow(preprocess(env.reset())))

In [ ]:
#Vamos fazer um stack de  4 frames pré-processados. Inicializamos este stack todo nulo
n_stacks=4 
stacked=deque([np.zeros((110,84), dtype=np.int) for i in range(n_stacks)], maxlen=4)

def stack_de_frames(stacked, state, indicador_de_episodio): #indicador_de_episodio é uma variável binária que indica se é ou não um novo episódio
#Primeiro vamos pré-processar:    
    processar=preprocess(state)
    
#Se for um novo episódio, limparemos o stack e faremos nosso novo stack:    
    if indicador_de_episodio:
#Limpa o stack:
        stacked=deque([np.zeros((110,84), dtype=np.int) for i in range(stack_size)], maxlen=4)
#Coloca o mesmo frame 4 vezes no stack em questão:
        stacked.append(state)
        stacked.append(state)
        stacked.append(state)
        stacked.append(state)
#Faz o stack via numpy:        
        stacked_state = np.stack(stacked, axis=2)

#Se não for um novo episódio, eliminamos o frame mais antigo do stack e colocamos nosso frame atual
    else:
        stacked.append(state)
        stacked_state = np.stack(stacked, axis=2)
    
    return stacked_state, stacked

## 2 - Seleção de Hiperparâmetros

In [ ]:
#Hiperparâmetros que poderemos ou não variar para avaliar a performance do nosso modelo
state_size = [110, 84, 4]      
action_size = env.action_space.n
learning_rate =  0.00025      
total_episodes = 50            
max_steps = 50000              
batch_size = 64                
explore_start = 1.0            
explore_stop = 0.01             
decay_rate = 0.00001           
gamma = 0.9                    
pretrain_length = batch_size   
memory_size = 1000000          
stack_size = 4                 
tau=0.05

## 3 - Desenvolvendo nossa Rede Neural

Em termos de arquitetura, nossa rede neural será composta da seguinte maneira:

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from keras.optimizers import Adam 

class DQNAgent():
    def __init__(self, env):
        self.memory = deque(maxlen=100000)
        self.env = env
        self.state_size = state_size
        self.action_size = action_size
        self.episodes = total_episodes
        self.env._max_episode_steps = max_steps
        self.epsilon = explore_start
        self.epsilon_decay = decay_rate
        self.epsilon_min = explore_stop
        self.gamma = gamma
        self.alpha = learning_rate
        self.batch_size = batch_size
        self.model = self.build_model()
        self.tau=tau
        
        
    def build_model(self):
        model = keras.Sequential()
        model.add(keras.Input(shape=(110,84,1)))
        model.add(Conv2D(filters = 32, kernel_size = [8,8], strides = [4,4], 
                         padding='valid',kernel_initializer="glorot_uniform"))
        model.add(Conv2D(filters = 64, kernel_size = [4,4], strides = [2,2], 
                         padding='valid',kernel_initializer="glorot_uniform"))
        model.add(Conv2D(filters = 64, kernel_size = [3,3], strides = [2,2], 
                         padding='valid',kernel_initializer="glorot_uniform"))
        model.add(Flatten())
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))
        return model
        
    def act(self, state):
        if (np.random.random() <= self.epsilon):
            return self.env.action_space.sample()
        return np.argmax(self.model.predict(state))
        
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
            
    def rreplay(self, batch_size):
        x_batch, y_batch = [], []
        minibatch = random.sample(self.memory, min(len(self.memory), batch_size))
        for state, action, reward, next_state, done in minibatch:
            y_target = self.model.predict(state)
            y_target[0][action] = reward if done else reward + self.gamma * np.max(self.model.predict(next_state)[0])
            x_batch.append(state[0])
            y_batch.append(y_target[0])
                
            self.model.fit(np.array(x_batch), np.array(y_batch), batch_size=len(x_batch), verbose=0)
            if self.epsilon > self.epsilon_min:
                self.epsilon *= self.epsilon_decay
    
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state)
            if done:
                target[0][action] = reward
            else:
                t = self.target_model.predict(next_state)[0]
                target[0][action] = reward + self.gamma * np.amax(t)
            self.model.fit(state, target, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        
    def target_train(self):
        #Construir função de treino
        
    
    def save_model(self, fn):
        self.model.save(fn)


In [ ]:
#Treinando e vendo nosso agente evoluir.
dqn_agent = DQNAgent(env=env)
steps = []

done=False
state=env.reset()
i=0

for i in range(0,total_episodes):
    print('geração {}'.format(i))
    state=env.reset()
    done=False
    while not done:
        state=preprocess(state)
        action = dqn_agent.act(state)
        next_state, reward, done, info = env.step(action)
        dqn_agent.remember(state, action, reward, next_state, done)
        state=next_state
        env.render()
        dqn_agent.target_train()
